In [2]:
from datasets import load_dataset
import os
os.environ['https_proxy'] = 'http://10.14.30.39:7890'
ds = load_dataset("Salesforce/wikitext", "wikitext-103-raw-v1")
ds

Using the latest cached version of the dataset since Salesforce/wikitext couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'wikitext-103-raw-v1' at /root/.cache/huggingface/datasets/Salesforce___wikitext/wikitext-103-raw-v1/0.0.0/b08601e04326c79dfdd32d625aee71d232d685c3 (last modified on Thu Nov 21 01:39:49 2024).


DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 1801350
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [3]:
train_ds = ds['train']
train_ds

Dataset({
    features: ['text'],
    num_rows: 1801350
})

In [4]:
len_mate_data = [len(i) for i in train_ds['text']]


import pandas as pd

def calculate_quartiles_with_pandas(data):
    """
    使用 Pandas 计算数据的四分位数
    :param data: 数据列表或数组
    :return: 第 1 四分位数、第 2 四分位数（中位数）、第 3 四分位数
    """
    series = pd.Series(data)
    q1 = series.quantile(0.25)  # 第 1 四分位数
    q2 = series.quantile(0.50)  # 中位数
    q3 = series.quantile(0.75)  # 第 3 四分位数
    return q1, q2, q3

# 示例用法
data = len_mate_data
q1, q2, q3 = calculate_quartiles_with_pandas(data)
print(f"第 1 四分位数: {q1}, 中位数: {q2}, 第 3 四分位数: {q3}")
print('mean:',sum(len_mate_data)/len(len_mate_data))

第 1 四分位数: 0.0, 中位数: 33.0, 第 3 四分位数: 557.0
mean: 298.82828600771643


In [6]:
train_ds = train_ds.filter(lambda x: len(x['text'])>300, num_proc=48)
train_ds = train_ds.filter(lambda x: len(x['text'])<400, num_proc=48)
train_ds

Filter (num_proc=48):   0%|          | 0/1801350 [00:00<?, ? examples/s]

Filter (num_proc=48):   0%|          | 0/650109 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 67517
})

In [7]:
from datasets import Dataset

train_ds_shu = train_ds.shuffle(seed=42)
dataset_used_A_train = train_ds_shu.select(range(0,20000))['text']
dataset_used_A_test = train_ds_shu.select(range(20000,22000))['text']
dataset_used_B_train_mem = train_ds_shu.select(range(22000,32000))['text']
dataset_used_B_train_non = train_ds_shu.select(range(32000,42000))['text']
dataset_used_B_test = train_ds_shu.select(range(42000,44000))['text']


import sys
sys.path.append('..')
from Data.utils import save_json,load_json

path = '../Data/raw/phishing/wikitext.json'

save_json({
    'A':{
        'train':dataset_used_A_train,
        'test':dataset_used_A_test
    },
    'B':{
        'train':dataset_used_B_train_mem,
        'test':dataset_used_B_test,
        'train_non':dataset_used_B_train_non
    }
},path)